In [84]:
import pandas as pd

# CSVファイルを読み込むが、データは読み込まない
df = pd.read_csv('/Users/shuma/Lecture/programing/dsprogramming-muro/nittere_news.csv', nrows=0)

# ヘッダー行を表示
print(df.columns.tolist())

# CSVファイルを開く
with open('/Users/shuma/Lecture/programing/dsprogramming-muro/nittere_news.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)

    # ヘッダー行をスキップする場合
    next(csv_reader)

    # 行ごとに読み込む
    for row in csv_reader:
        print(row)

['day', 'event']
['1月1日', '能登半島地震の警告']
['1月2日', 'JALの炎上事故救助活動Live中継']
['1月3日', '海保機の侵入の許可なしと国交省が通信記録を発表']
['1月4日', '菅制官の「海保機の侵入に気づかなかった」という説明']
['1月5日', 'カメラが捉えた衝突直前の海保機、一部始終']
['1月6日', '避難生活の取材']
['1月7日', '岸田首相が池田議員の逮捕を受けコメント']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']
['', '']


In [85]:
import pandas as pd
import sqlite3

# CSVファイルのパス
csv_file = '/Users/shuma/Lecture/programing/dsprogramming-muro/nittere_news.csv'

# SQLiteデータベースファイルのパス（存在しない場合は新規作成されます）
db_file = 'example.db'

# CSVファイルをデータフレームとして読み込む
df = pd.read_csv(csv_file)

# SQLiteデータベースに接続
conn = sqlite3.connect(db_file)

# データフレームをデータベースのテーブルに保存（テーブル名は 'news_table' とします）
# 既にテーブルが存在する場合は置き換える
df.to_sql('news_table', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()


In [86]:
pip install pandas sqlalchemy scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [87]:
from sqlalchemy import create_engine
import pandas as pd

# データベース接続設定（例：SQLiteの場合）
engine1 = create_engine('sqlite:///example.db')
engine2 = create_engine('sqlite:///news.db')

In [88]:
# データベースからデータを読み込む
data1 = pd.read_sql('SELECT * FROM news_table', engine1)
data2 = pd.read_sql('SELECT * FROM news', engine2)

In [89]:
print(data1.columns)
print(data2.columns)

Index(['day', 'event'], dtype='object')
Index(['date', 'title', 'link'], dtype='object')


In [90]:
data1['event'] = data1['event'].fillna('')
data2['title'] = data2['title'].fillna('')


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDFベクトル化
vectorizer = TfidfVectorizer()

# 'event' 列と 'title' 列を使用してベクトル化
vectors = vectorizer.fit_transform(data1['event'].tolist() + data2['title'].tolist())

# ベクトルをそれぞれのデータセットに分割
vectors1 = vectors[:len(data1)]
vectors2 = vectors[len(data1):]



In [92]:
from sklearn.metrics.pairwise import cosine_similarity

# データセット間でコサイン類似度を計算
similarity_matrix = cosine_similarity(vectors1, vectors2)


In [93]:
print(similarity_matrix)


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [94]:
import numpy as np

# 類似度が最も高いペアのトップNを表示
N = 5
top_indices = np.dstack(np.unravel_index(np.argsort(-similarity_matrix.ravel()), similarity_matrix.shape))[0][:N]

for i, j in top_indices:
    print(f"文書 {i} (data1) と 文書 {j} (data2) の類似度: {similarity_matrix[i, j]}")


文書 0 (data1) と 文書 0 (data2) の類似度: 0.0
文書 10 (data1) と 文書 0 (data2) の類似度: 0.0
文書 9 (data1) と 文書 6 (data2) の類似度: 0.0
文書 9 (data1) と 文書 5 (data2) の類似度: 0.0
文書 9 (data1) と 文書 4 (data2) の類似度: 0.0
